In [83]:
import pandas as pd
import random
import numpy as np
from random import randint
import torch
from transformers import AutoTokenizer, AutoModel
import mteb
from sentence_transformers import SentenceTransformer

# import adapters
# from adapters import AutoAdapterModel
# import gc

# import scipy as sp
# from scipy import sparse
# import sklearn

# import matplotlib.pyplot as plt
# import matplotlib as mpl

import os
import pickle
import time
import memory_profiler
from tqdm.notebook import tqdm

%load_ext memory_profiler

from pathlib import Path
import distro

%load_ext watermark

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


In [2]:
# old one '1.8.1+cu111'
torch.__version__

'2.1.1+cu121'

In [4]:
%load_ext autoreload
%autoreload 2

from text_embeddings_src.model_stuff import train_loop
from text_embeddings_src.data_stuff import (
    MultOverlappingSentencesPairDataset,
)
from text_embeddings_src.embeddings import generate_embeddings

# from text_embeddings_src.metrics import knn_accuracy, logistic_accuracy
# from text_embeddings_src.dim_red import run_tsne_simple
# from text_embeddings_src.plotting import plot_tsne_colors

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import black
import jupyter_black

jupyter_black.load(line_length=79)

In [6]:
variables_path = Path("../../results/variables")
# variables_pubmed_path = Path("../../pubmed-landscape/results/variables")
figures_path = Path("../../results/figures/updated_dataset")
data_path = Path("../../data")
berenslab_data_path = Path("/gpfs01/berens/data/data/pubmed_processed")

In [7]:
# MANUAL FIX TO PATH ISSUE FROM VSCODE
import text_embeddings_src

nb_path = Path(text_embeddings_src.__path__[0]).parents[0] / Path(
    "scripts/updated_dataset"
)
assert nb_path.exists(), "The path does not exist"

variables_path = (nb_path / variables_path).resolve(strict=True)
figures_path = (nb_path / figures_path).resolve(strict=True)
data_path = (nb_path / data_path).resolve(strict=True)

In [8]:
# plt.style.use((nb_path / Path("../matplotlib_style.txt")).resolve(strict=True))

In [9]:
%watermark -a 'Rita González-Márquez' -t -d -tz -u -v -iv -w -m -h -p transformers -p openTSNE
print(distro.name(pretty=True))

Author: Rita González-Márquez

Last updated: 2024-09-17 09:14:39CEST

Python implementation: CPython
Python version       : 3.11.5
IPython version      : 8.18.1

openTSNE: 1.0.0

Compiler    : GCC 11.2.0
OS          : Linux
Release     : 3.10.0-1160.el7.x86_64
Machine     : x86_64
Processor   : x86_64
CPU cores   : 64
Architecture: 64bit

Hostname: rgonzalesmarquez_GPU0-llm_gber7

memory_profiler    : 0.61.0
torch              : 2.1.1
mteb               : 1.12.91
jupyter_black      : 0.3.4
text_embeddings_src: 0.0.0
numpy              : 1.26.2
pandas             : 2.1.3
black              : 23.11.0
distro             : 1.8.0

Watermark: 2.4.3

Ubuntu 22.04.3 LTS


# Import ICLR dataset

In [32]:
%%time
iclr2024 = pd.read_parquet(
    data_path / "iclr2024.parquet.gzip",
    # index=False,
    engine="pyarrow",
    # compression="gzip",
)

CPU times: user 294 ms, sys: 111 ms, total: 405 ms
Wall time: 753 ms


In [33]:
iclr2024.keywords = iclr2024.keywords.transform(lambda x: list(x))
iclr2024.scores = iclr2024.scores.transform(lambda x: list(x))

In [34]:
iclr2024

,index,year,id,title,abstract,authors,decision,scores,keywords,gender-first,gender-last,t-SNE x,t-SNE y
0,0,2017,S1VaB4cex,FractalNet: Ultra-Deep Neural Networks without...,We introduce a design strategy for neural netw...,"Gustav Larsson, Michael Maire, Gregory Shakhna...",Accept (Poster),"[5, 7, 6, 6]",[],male,male,-28.117955,-20.418127
1,1,2017,H1W1UN9gg,Deep Information Propagation,We study the behavior of untrained neural netw...,"Samuel S. Schoenholz, Justin Gilmer, Surya Gan...",Accept (Poster),"[8, 9, 8]","[theory, deep learning]",male,None,-32.466820,-10.791123
2,2,2017,r1GKzP5xx,Recurrent Normalization Propagation,We propose a LSTM parametrization that preser...,"César Laurent, Nicolas Ballas, Pascal Vincent",Invite to Workshop Track,"[4, 6, 6]","[deep learning, optimization]",None,male,3.504240,19.946053
3,3,2017,S1J0E-71l,Surprisal-Driven Feedback in Recurrent Networks,Recurrent neural nets are widely used for pred...,"K, a, m, i, l, , R, o, c, k, i",Reject,"[3, 4, 3]","[unsupervised learning, applications, deep lea...",None,None,4.553473,16.037763
4,4,2017,SJGCiw5gl,Pruning Convolutional Neural Networks for Reso...,We propose a new formulation for pruning convo...,"Pavlo Molchanov, Stephen Tyree, Tero Karras, T...",Accept (Poster),"[6, 7, 9]","[deep learning, transfer learning]",None,male,-25.827705,-37.891772
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24342,7299,2024,1bbPQShCT2,I-PHYRE: Interactive Physical Reasoning,Current evaluation protocols predominantly ass...,,,[],"[intuitive physics, physical reasoning]",None,None,43.137120,44.316133
24343,7300,2024,Ny150AblPu,EXPOSING TEXT-IMAGE INCONSISTENCY USING DIFFUS...,In the battle against widespread online misinf...,,,[],"[mis-contextualization, media forensic]",None,None,59.742172,-22.673627
24344,7301,2024,ZGBOfAQrMl,Video Super-Resolution Transformer with Masked...,"Recently, Vision Transformer has achieved grea...",,,[],"[video super-resolution, adaptive, memory and ...",None,None,57.933273,-3.932825
24345,7302,2024,J2kRjUAOLh,Contrastive Predict-and-Search for Mixed Integ...,Mixed integer linear programs (MILP) are flex...,,,[],[mixed integer programs; contrastive learning],None,None,-11.437999,21.289523


In [35]:
labels_iclr = np.load(variables_path / "updated_dataset" / "labels_iclr.npy")
colors_iclr = np.load(variables_path / "updated_dataset" / "colors_iclr.npy")

pickle_in = open(
    variables_path / "updated_dataset" / "dict_label_to_color.pkl", "rb"
)
dict_label_to_color = pickle.load(pickle_in)

In [36]:
# sanity check
print(len(np.unique(labels_iclr)))
labels_iclr

46


array(['unlabeled', 'unlabeled', 'optimization', ..., 'unlabeled',
       'unlabeled', 'federated learning'], dtype='<U34')

# Functions

In [10]:
def fix_all_seeds(seed=42):
    # Set the random seed for PyTorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  ## this one is new
    ## Set the seed for generating random numbers on all GPUs.
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    # torch.use_deterministic_algorithms(True) ## this one I don't use but don't remember why

    # Set the random seed for NumPy
    np.random.seed(seed)

    # Set the random seed
    random.seed(seed)

## Function train model without evaluation

In [15]:
import torch
from tqdm.notebook import tqdm
from transformers.optimization import get_linear_schedule_with_warmup
from text_embeddings_src.model_stuff import mean_pool


def train_loop_without_eval(
    model,
    loader,
    device,
    optimized_rep="av",
    n_epochs=1,
    lr=2e-5,
):
    assert optimized_rep in [
        "av",
        "cls",
        "sep",
        "7th",
    ], "Not valid `optimized_rep`. Choose from ['av', 'cls', 'sep', '7th']."

    model.to(device)

    # define layers to be used in multiple-negatives-ranking
    cos_sim = torch.nn.CosineSimilarity()
    loss_func = torch.nn.CrossEntropyLoss()
    scale = 20.0  # we multiply similarity score by this scale value, it is the inverse of the temperature
    # move layers to device
    cos_sim.to(device)
    loss_func.to(device)

    # initialize Adam optimizer
    optim = torch.optim.Adam(model.parameters(), lr=lr)

    # setup warmup for first ~10% of steps
    total_steps = len(loader) * n_epochs
    warmup_steps = int(0.1 * len(loader))
    scheduler = get_linear_schedule_with_warmup(
        optim,
        num_warmup_steps=warmup_steps,
        num_training_steps=total_steps,
    )

    losses = np.empty((n_epochs, len(loader)))
    accuracies = []
    for epoch in range(n_epochs):
        model.train()  # make sure model is in training mode
        # initialize the dataloader loop with tqdm (tqdm == progress bar)
        loop = tqdm(loader, leave=True)
        for i_batch, batch in enumerate(loop):
            # zero all gradients on each new step
            optim.zero_grad()
            # prepare batches and move all to the active device
            anchor_ids = batch[0][0].to(
                device
            )  # this are all anchor abstracts from the batch,len(anchor_ids)= len(batch)
            anchor_mask = batch[0][1].to(device)
            pos_ids = batch[1][0].to(
                device
            )  # this each positive pair from each anchor, all in one array, also len(batch)
            pos_mask = batch[1][1].to(device)
            # extract token embeddings from BERT
            a = model(anchor_ids, attention_mask=anchor_mask)[
                0
            ]  # all token embeddings
            p = model(pos_ids, attention_mask=pos_mask)[0]

            # get the mean pooled vectors  -- put all of these ifs into a pool function (wraper) to which I pass, a, p the masks and the optimized rep
            if optimized_rep == "av":
                a = mean_pool(a, anchor_mask)
                p = mean_pool(p, pos_mask)

            elif optimized_rep == "cls":
                a = cls_pool(a, anchor_mask)
                p = cls_pool(p, pos_mask)

            elif optimized_rep == "sep":
                a = sep_pool(a, anchor_mask)
                p = sep_pool(p, pos_mask)

            elif optimized_rep == "7th":
                a = seventh_pool(a, anchor_mask)
                p = seventh_pool(p, pos_mask)

            # calculate the cosine similarities
            scores = torch.stack(
                [cos_sim(a_i.reshape(1, a_i.shape[0]), p) for a_i in a]
            )
            # get label(s) - we could define this before if confident
            # of consistent batch sizes
            labels = torch.tensor(
                range(len(scores)), dtype=torch.long, device=scores.device
            )  # I think that the labels are just the label of which pair it is. 0 for the first pair, 1 for the second...
            # my guess is that they are used in the loss to know which of the cosine similarities should be high
            # and which low

            # and now calculate the loss
            loss = loss_func(scores * scale, labels)
            losses[epoch, i_batch] = loss.item()

            # using loss, calculate gradients and then optimize
            loss.backward()
            optim.step()
            # update learning rate scheduler
            scheduler.step()
            # update the TDQM progress bar
            loop.set_description(f"Epoch {epoch}")
            loop.set_postfix(loss=loss.item())

        return model, losses

# Fine-tune MPNet
We repeat the standard finetuning to be able to save the checkpoints

In [16]:
model_names = [
    "MPNet",
]


model_paths = [
    "microsoft/mpnet-base",
]

In [17]:
%%time

# fix random seeds
fix_all_seeds()

# set up model
print("Model: ", model_names[0])

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on device: {}".format(device))

tokenizer = AutoTokenizer.from_pretrained(model_paths[0])
model = AutoModel.from_pretrained(model_paths[0])


# data
training_dataset = MultOverlappingSentencesPairDataset(
    iclr2024.abstract,
    tokenizer,
    device,
)

gen = torch.Generator()
gen.manual_seed(42)
training_loader = torch.utils.data.DataLoader(
    training_dataset, batch_size=64, shuffle=True, generator=gen
)

# training
model_trained_crops, losses = train_loop_without_eval(
    model,
    training_loader,
    device,
    optimized_rep="av",
    n_epochs=1,
    lr=2e-5,
)

# Save the model
output_dir = Path("updated_dataset/model_checkpoints/mpnet_crops_finetuning")
model_trained_crops.save_pretrained(variables_path / output_dir)


print("----------------------------")

Model:  MPNet
Running on device: cuda


/gpfs01/berens/user/rgonzalesmarquez/.local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of MPNetModel were not initialized from the model checkpoint at microsoft/mpnet-base and are newly initialized: ['mpnet.pooler.dense.weight', 'mpnet.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


microsoft/mpnet-base


  0%|          | 0/368 [00:00<?, ?it/s]

----------------------------
CPU times: user 6min 6s, sys: 4min 5s, total: 10min 12s
Wall time: 5min 15s


# Benchmarks

## Task 
ArxivClusteringP2P
BiorxivClusteringP2P
MedrxivClusteringP2P
RedditClusteringP2P
TwentyNewsgroupsClustering
SciDocsRR
MindSmall
SCIDOCS 
ArguAna
STS12
STS13
STS14
STS15
STS16
STSBenchmark

In [10]:
# ["ArxivClusteringP2P", "BiorxivClusteringP2P", "MedrxivClusteringP2P", "RedditClusteringP2P", "TwentyNewsgroupsClustering", "SciDocsRR", "MindSmall", "SCIDOCS ", "ArguAna", "STS12", "STS13", "STS14", "STS15", "STS16", "STSBenchmark"]

In [11]:
# tasks = mteb.get_tasks(tasks=["ArxivClusteringP2P"])
# evaluation = mteb.MTEB(tasks=tasks)

In [11]:
tasks = mteb.get_tasks(
    tasks=[
        # "ArxivClusteringP2P",
        # "BiorxivClusteringP2P",
        # "MedrxivClusteringP2P",
        # "RedditClusteringP2P",
        # "TwentyNewsgroupsClustering",
        "StackExchangeClusteringP2P",
        # "SciDocsRR",
        # "MindSmallReranking",
        # "SCIDOCS",
        # "ArguAna",
        # "STS12",
        # "STS13",
        # "STS14",
        # "STS15",
        # "STS16",
        # "STSBenchmark",
    ]
)
evaluation = mteb.MTEB(tasks=tasks)

### Pre-trained MPNet

In [12]:
model_names = [
    "MPNet",
]


model_paths = [
    "microsoft/mpnet-base",
]

In [13]:
# pretrained_mpnet = AutoModel.from_pretrained(model_paths[0])
pretrained_mpnet = SentenceTransformer(model_paths[0])

Some weights of MPNetModel were not initialized from the model checkpoint at microsoft/mpnet-base and are newly initialized: ['mpnet.pooler.dense.bias', 'mpnet.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
pretrained_mpnet

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [15]:
saving_path = (
    Path("embeddings_" + model_names[0].lower())
    / Path("updated_dataset")
    / Path("mteb_benchmark")
)
(variables_path / saving_path).mkdir(exist_ok=True)

results = evaluation.run(
    pretrained_mpnet,
    output_folder=variables_path
    / saving_path
    / f"results_{model_names[0].lower()}",
)

───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

Clustering

- StackExchangeClusteringP2P, p2p

Clustering: 100%|██████████████████████████████████████████████████████████████| 10/10 [06:37<00:00, 39.76s/it]


In [16]:
print(results[0].scores["test"][0]["main_score"])

0.26347092269579386


#### test to load the results

In [33]:
import os
import json


def load_mteb_results(directory):
    results = {}
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".json"):
                file_path = os.path.join(root, file)
                with open(file_path, "r") as f:
                    data = json.load(f)

                # Create a key based on the relative path
                relative_path = os.path.relpath(file_path, directory)
                key = os.path.splitext(relative_path)[
                    0
                ]  # Remove the .json extension
                results[key] = data

    return results

In [35]:
root_directory = variables_path / saving_path / "results_mpnet"
mteb_results_mpnet = load_mteb_results(root_directory)

In [43]:
mteb_results_mpnet[
    "microsoft__mpnet-base/6996ce1e91bd2a9c7d7f61daec37463394f73f09/ArxivClusteringP2P"
]["scores"]["test"][0]["main_score"]

0.2779650469944176

### Fine-tuned MPNet (crops)

In [17]:
model_names = [
    "MPNet",
]


model_paths = [
    "microsoft/mpnet-base",
]

In [18]:
# SECOND
from sentence_transformers import models
from sentence_transformers.models import Transformer

# Load your custom base model and tokenizer
checkpoint_dir = Path(
    "updated_dataset/model_checkpoints/mpnet_crops_finetuning"
)
custom_base_model = AutoModel.from_pretrained(variables_path / checkpoint_dir)

custom_tokenizer = AutoTokenizer.from_pretrained("microsoft/mpnet-base")

# Create a new SentenceTransformer model
new_modules = []

# Wrap your custom base model in a Transformer module
transformer_model = models.Transformer(
    model_name_or_path="bert-base-uncased",  # None gives an error,
    # so I initialize another model that
    # will be substituted by my custom model below
    max_seq_length=384,  # You can adjust this as needed
    do_lower_case=False,  # Adjust based on your tokenizer
)
# Replace the auto_model in the Transformer wrapper with your custom model
transformer_model.auto_model = custom_base_model
# Set the tokenizer
transformer_model.tokenizer = custom_tokenizer

# Add the wrapped model as the first module
new_modules.append(transformer_model)

# Add Pooling layer
pooling_model = models.Pooling(
    word_embedding_dimension=custom_base_model.config.hidden_size,
    pooling_mode_cls_token=False,
    pooling_mode_mean_tokens=True,
    pooling_mode_max_tokens=False,
    pooling_mode_mean_sqrt_len_tokens=False,
)
new_modules.append(pooling_model)

# Add Normalize layer
new_modules.append(models.Normalize())

# Create the new SentenceTransformer model
finetuned_mpnet = SentenceTransformer(modules=new_modules)

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [19]:
model_name = "MPNet"
saving_path = (
    Path("embeddings_" + model_name.lower())
    / Path("updated_dataset")
    / Path("mteb_benchmark")
)
(variables_path / saving_path).mkdir(exist_ok=True)

results = evaluation.run(
    finetuned_mpnet,
    output_folder=variables_path
    / saving_path
    / f"results_{model_name.lower()}_crops_finetuning",
)

───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

Clustering

- StackExchangeClusteringP2P, p2p

Clustering:  90%|████████████████████████████████████████████████████████▋      | 9/10 [05:17<00:27, 27.56s/it]

In [65]:
print(results[0].scores["test"][0]["main_score"])

0.3831448182125486


#### Another way of doing it -- Does not work
The kernel crashes for some unknown reason

In [ ]:
# import os
# import torch
# import numpy as np
# from lightning.fabric import Fabric
# from transformers import AutoModel, AutoTokenizer
# from torch.nn import functional as F
# from mteb import MTEB

# def average_pool(last_hidden_states, attention_mask):
#     last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
#     return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]


# class EmbModel:
#     def __init__(self):
#         self.tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-base')
#         self.model = AutoModel.from_pretrained('intfloat/multilingual-e5-base')

#     @torch.inference_mode()
#     def encode(self, input_texts, batch_size=None, **kwargs):
#         batch_dict = self.tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')
#         outputs = self.model(**batch_dict)
#         embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
#         embeddings = F.normalize(embeddings, p=2, dim=1)
#         return embeddings.cpu().numpy()

# fabric = Fabric(devices=1, accelerator='cuda', strategy='deepspeed_stage_2')
# fabric.launch()

# model = EmbModel()
# assert isinstance(model.encode(['Hello world', 'foo bar']), np.ndarray)
# print('RANK is set to', os.getenv('RANK'))
# evaluation = MTEB(tasks=["TRECCOVID"])
# results = evaluation.run(model)

In [10]:
def mean_pool(token_embeds, attention_mask):
    # reshape attention_mask to cover 768-dimension embeddings
    in_mask = attention_mask.unsqueeze(-1).expand(token_embeds.size())
    # perform mean-pooling but exclude padding tokens (specified by in_mask)
    pool = torch.sum(token_embeds * in_mask, 1) / torch.clamp(
        in_mask.sum(1), min=1e-9
    )
    return pool


def cls_pool(token_embeds, attention_mask):
    ix0 = torch.arange(attention_mask.size(0))
    return token_embeds[ix0, 0, :]


class MyModel:
    def __init__(self, model_path, tokenizer_path=None, pooling_method="av"):
        assert pooling_method in [
            "av",
            "cls",
        ], "Not valid `pooling_method`. Choose from ['av', 'cls']."

        if tokenizer_path == None:
            tokenizer_path = model_path

        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
        self.model = AutoModel.from_pretrained(model_path)
        # self.device = (
        #     torch.device("cuda")
        #     if torch.cuda.is_available()
        #     else torch.device("cpu")
        # )
        self.pooling_method = pooling_method

        # self.model.to(self.device)

    @torch.no_grad()  # what is the difference between no_grad() and inference_mode()?
    def encode(self, input_texts, batch_size=None, **kwargs):
        batch_dict = self.tokenizer(
            input_texts,
            max_length=512,
            padding=True,
            truncation=True,
            return_tensors="pt",
        )  # .to(self.device)
        outputs = self.model(**batch_dict)
        if self.pooling_method == "av":
            embeddings = mean_pool(
                outputs.last_hidden_state, batch_dict["attention_mask"]
            )
        elif self.pooling_method == "cls":
            embeddings = cls_pool(
                outputs.last_hidden_state, batch_dict["attention_mask"]
            )
        return embeddings  # .detach().cpu().numpy()

In [11]:
# Load your custom base model and tokenizer
checkpoint_dir = Path(
    "updated_dataset/model_checkpoints/mpnet_crops_finetuning"
)
finetuned_mpnet_v2 = MyModel(
    model_path=(variables_path / checkpoint_dir),
    tokenizer_path="microsoft/mpnet-base",
)

/gpfs01/berens/user/rgonzalesmarquez/.local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [12]:
finetuned_mpnet_v2

In [13]:
model_name = "MPNet"
saving_path = (
    Path("embeddings_" + model_name.lower())
    / Path("updated_dataset")
    / Path("mteb_benchmark")
)
(variables_path / saving_path).mkdir(exist_ok=True)

results = evaluation.run(
    finetuned_mpnet_v2,
    output_folder=variables_path
    / saving_path
    / f"results_{model_name.lower()}_crops_finetuning_v2",
)

───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

Clustering

- ArxivClusteringP2P, p2p

Clustering:   0%|                                                                       | 0/31 [00:00<?, ?it/s]

: 

: 

: 

#### Yet another way -- Does not work either

In [15]:
# pretrained_mpnet = AutoModel.from_pretrained(model_paths[0])
checkpoint_dir = Path(
    "updated_dataset/model_checkpoints/mpnet_crops_finetuning"
)
finetuned_mpnet_v2 = SentenceTransformer(
    "../../results/variables/updated_dataset/model_checkpoints/mpnet_crops_finetuning"
)

ValueError: Path ../../results/variables/updated_dataset/model_checkpoints/mpnet_crops_finetuning not found

In [14]:
variables_path / checkpoint_dir

PosixPath('../../results/variables/updated_dataset/model_checkpoints/mpnet_crops_finetuning')

### Fine-tuned MPNet (SimCSE-like)

In [ ]:
# SECOND
from sentence_transformers import models
from sentence_transformers.models import Transformer

# Load your custom base model and tokenizer
checkpoint_dir = Path(
    "updated_dataset/model_checkpoints/mpnet_simcse_finetuning"
)
custom_base_model = AutoModel.from_pretrained(variables_path / checkpoint_dir)

custom_tokenizer = AutoTokenizer.from_pretrained("microsoft/mpnet-base")

# Create a new SentenceTransformer model
new_modules = []

# Wrap your custom base model in a Transformer module
transformer_model = models.Transformer(
    model_name_or_path="bert-base-uncased",  # None gives an error,
    # so I initialize another model that
    # will be substituted by my custom model below
    max_seq_length=384,  # You can adjust this as needed
    do_lower_case=False,  # Adjust based on your tokenizer
)
# Replace the auto_model in the Transformer wrapper with your custom model
transformer_model.auto_model = custom_base_model
# Set the tokenizer
transformer_model.tokenizer = custom_tokenizer

# Add the wrapped model as the first module
new_modules.append(transformer_model)

# Add Pooling layer
pooling_model = models.Pooling(
    word_embedding_dimension=custom_base_model.config.hidden_size,
    pooling_mode_cls_token=False,
    pooling_mode_mean_tokens=True,
    pooling_mode_max_tokens=False,
    pooling_mode_mean_sqrt_len_tokens=False,
)
new_modules.append(pooling_model)

# Add Normalize layer
new_modules.append(models.Normalize())

# Create the new SentenceTransformer model
finetuned_mpnet_simcse = SentenceTransformer(modules=new_modules)

In [ ]:
model_name = "MPNet"
saving_path = (
    Path("embeddings_" + model_name.lower())
    / Path("updated_dataset")
    / Path("mteb_benchmark")
)
(variables_path / saving_path).mkdir(exist_ok=True)

results = evaluation.run(
    finetuned_mpnet_simcse,
    output_folder=variables_path
    / saving_path
    / f"results_{model_name.lower()}_simcse_finetuning",
)

───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

Retrieval

- SCIDOCS, s2p

- ArguAna, s2p

STS

...

Batches:   0%|          | 0/894 [00:00<?, ?it/s]

Clustering:  60%|█████████████████████████████████████▏                        | 6/10 [12:15<08:45, 131.48s/it]

Batches:   0%|          | 0/2161 [00:00<?, ?it/s]

Clustering:  70%|███████████████████████████████████████████▍                  | 7/10 [14:33<06:41, 133.80s/it]

Batches:   0%|          | 0/2109 [00:00<?, ?it/s]

### Pre-trained SBERT

In [ ]:
model_names = [
    "SBERT",
]


model_paths = [
    "sentence-transformers/all-mpnet-base-v2",
]

In [ ]:
# pretrained_sbert = AutoModel.from_pretrained(
#     "sentence-transformers/all-mpnet-base-v2"
# )  # doesn't work

pretrained_sbert = SentenceTransformer(
    "sentence-transformers/all-mpnet-base-v2"
)

In [ ]:
pretrained_sbert

In [ ]:
saving_path = (
    Path("embeddings_" + model_names[0].lower())
    / Path("updated_dataset")
    / Path("mteb_benchmark")
)
(variables_path / saving_path).mkdir(exist_ok=True)

results = evaluation.run(
    pretrained_sbert,
    output_folder=variables_path
    / saving_path
    / f"results_{model_names[0].lower()}",
)

In [56]:
results[0].scores  # scores

{'test': [{'v_measure': 0.48122865602904,
   'v_measure_std': 0.1420737036434218,
   'v_measures': [0.47747775788196556,
    0.46992600355904723,
    0.4827140346059144,
    0.4885767563767572,
    0.4894702123648364,
    0.48157088445711504,
    0.4622690570510022,
    0.47304447734321053,
    0.4808516368387714,
    0.479967635923842,
    0.5684267022079372,
    0.5710113196939407,
    0.5681439209719611,
    0.5706508654041879,
    0.5695516421163074,
    0.5742516683510883,
    0.5706572886532014,
    0.5693702625249434,
    0.564890036535079,
    0.5707430582833042,
    0.529674654063362,
    0.2553072011727571,
    0.4713293397649489,
    0.41335919030567564,
    0.3528146126776203,
    0.28343364210120076,
    0.3028504710337586,
    0.22292767529384594,
    0.318104027431572,
    1.0,
    0.284722301911086],
   'main_score': 0.48122865602904,
   'hf_subset': 'default',
   'languages': ['eng-Latn']}]}

### Pre-trained SimCSE

In [11]:
model_names = [
    "SimCSE",
]


model_paths = [
    "princeton-nlp/unsup-simcse-bert-base-uncased",
]

In [12]:
# pretrained_sbert = AutoModel.from_pretrained(
#     "sentence-transformers/all-mpnet-base-v2"
# )  # doesn't work

pretrained_simcse = SentenceTransformer(model_paths[0])

/gpfs01/berens/user/rgonzalesmarquez/.local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [13]:
pretrained_simcse

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [14]:
saving_path = (
    Path("embeddings_" + model_names[0].lower())
    / Path("updated_dataset")
    / Path("mteb_benchmark")
)
(variables_path / saving_path).mkdir(exist_ok=True)

results = evaluation.run(
    pretrained_simcse,
    output_folder=variables_path
    / saving_path
    / f"results_{model_names[0].lower()}",
)

───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

Reranking

- SciDocsRR, s2s

- MindSmallReranking, s2s

Retrieval

...

Batches:   0%|          | 0/94 [00:00<?, ?it/s]

Batches:   0%|          | 0/38 [00:00<?, ?it/s]

Batches:   0%|          | 0/38 [00:00<?, ?it/s]

Batches:   0%|          | 0/44 [00:00<?, ?it/s]

Batches:   0%|          | 0/44 [00:00<?, ?it/s]

In [31]:
results[0].scores  # scores

{'test': [{'v_measure': 0.28807582656487557,
   'v_measure_std': 0.016650520720206934,
   'v_measures': [0.2780986581221716,
    0.2696831576885149,
    0.2692357874378456,
    0.26793095576309434,
    0.27492677329510967,
    0.3124345646729936,
    0.3016249511045954,
    0.2992775441140408,
    0.30546110723744707,
    0.302084766212942],
   'main_score': 0.28807582656487557,
   'hf_subset': 'default',
   'languages': ['eng-Latn']}]}

### Random embedding module

#### Test set up model

##### Way 1

In [ ]:
class EmbeddingOnlyModel(torch.nn.Module):
    def __init__(self, embeddings_module):
        super().__init__()
        self.embeddings = embeddings_module

    def forward(self, input_ids, position_ids=None, inputs_embeds=None):
        return self.embeddings(input_ids, position_ids, inputs_embeds)

In [ ]:
from transformers import BertConfig, BertModel

# Initializing a BERT bert-base-uncased style configuration
configuration = BertConfig()

# Initializing a model (with random weights) from the bert-base-uncased style configuration
model_random = BertModel(configuration)

# Accessing the model configuration
configuration = model_random.config

In [ ]:
%%time

# fix random seeds
fix_all_seeds()

# set up model
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on device: {}".format(device))

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
## model
# Load the pretrained model
# pretrained_model = AutoModel.from_pretrained(model_paths[0])
# Extract the embedding layer
embedding_layer = model_random.embeddings  # .word_embeddings
# new model
embedding_only_model = EmbeddingOnlyModel(embedding_layer)

Running on device: cuda
CPU times: user 29.2 ms, sys: 9.68 ms, total: 38.9 ms
Wall time: 151 ms


/gpfs01/berens/user/rgonzalesmarquez/.local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
# SECOND
from sentence_transformers import models
from sentence_transformers.models import Transformer

# Load your custom base model and tokenizer
checkpoint_dir = Path(
    "updated_dataset/model_checkpoints/mpnet_crops_finetuning"
)
custom_base_model = AutoModel.from_pretrained(variables_path / checkpoint_dir)

custom_tokenizer = AutoTokenizer.from_pretrained("microsoft/mpnet-base")

# Create a new SentenceTransformer model
new_modules = []

# Wrap your custom base model in a Transformer module
transformer_model = models.Transformer(
    model_name_or_path="bert-base-uncased",  # None gives an error,
    # so I initialize another model that
    # will be substituted by my custom model below
    max_seq_length=384,  # You can adjust this as needed
    do_lower_case=False,  # Adjust based on your tokenizer
)
# Replace the auto_model in the Transformer wrapper with your custom model
transformer_model.auto_model = custom_base_model
# Set the tokenizer
transformer_model.tokenizer = custom_tokenizer

# Add the wrapped model as the first module
new_modules.append(transformer_model)

# Add Pooling layer
pooling_model = models.Pooling(
    word_embedding_dimension=custom_base_model.config.hidden_size,
    pooling_mode_cls_token=False,
    pooling_mode_mean_tokens=True,
    pooling_mode_max_tokens=False,
    pooling_mode_mean_sqrt_len_tokens=False,
)
new_modules.append(pooling_model)

# Add Normalize layer
new_modules.append(models.Normalize())

# Create the new SentenceTransformer model
finetuned_mpnet = SentenceTransformer(modules=new_modules)

/gpfs01/berens/user/rgonzalesmarquez/.local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


##### Way 2 -- good one

In [ ]:
def mean_pool(token_embeds, attention_mask):
    # reshape attention_mask to cover 768-dimension embeddings
    in_mask = attention_mask.unsqueeze(-1).expand(token_embeds.size())
    # perform mean-pooling but exclude padding tokens (specified by in_mask)
    pool = torch.sum(token_embeds * in_mask, 1) / torch.clamp(
        in_mask.sum(1), min=1e-9
    )
    return pool


def cls_pool(token_embeds, attention_mask):
    ix0 = torch.arange(attention_mask.size(0))
    return token_embeds[ix0, 0, :]

import datasets
class MyEmbeddingSentenceModel:
    def __init__(self, model, tokenizer, pooling_method="av"):
        assert pooling_method in [
            "av",
            "cls",
        ], "Not valid `pooling_method`. Choose from ['av', 'cls']."

        self.tokenizer = tokenizer
        self.model = model
        self.device = (
            torch.device("cuda")
            if torch.cuda.is_available()
            else torch.device("cpu")
        )
        self.pooling_method = pooling_method

        self.model.to(self.device)

    @torch.no_grad()  # what is the difference between no_grad() and inference_mode()?
    def encode(self, input_texts, batch_size=None, **kwargs):
        inputs = self.tokenizer(
            input_texts,
            max_length=512,
            padding=True,
            truncation=True,
            return_tensors="pt",
        ).to(self.device)

        dataset = datasets.Dataset.from_dict(inputs)
        dataset.set_format(type="torch", output_all_columns=True)
        loader = torch.utils.data.DataLoader(
            dataset, batch_size=batch_size, num_workers=10
        )
        embeddings = []
        with torch.no_grad():
            for batch in loader:
                batch = {k: v.to(device) for k, v in batch.items()}
                outputs = self.model(batch.input_ids)
                if self.pooling_method == "av":
                    embdd = mean_pool(outputs, batch_dict["attention_mask"])
                elif self.pooling_method == "cls":
                    embdd = cls_pool(outputs, batch_dict["attention_mask"])
                embeddings.append(embdd)
                
        embeddings = np.vstack(embeddings)
        return embeddings  # .detach().cpu().numpy()

In [ ]:
class EmbeddingOnlyModel(torch.nn.Module):
    def __init__(self, embeddings_module):
        super().__init__()
        self.embeddings = embeddings_module

    def forward(self, input_ids, position_ids=None, inputs_embeds=None):
        return self.embeddings(input_ids, position_ids, inputs_embeds)

In [ ]:
from transformers import BertConfig, BertModel

# Initializing a BERT bert-base-uncased style configuration
configuration = BertConfig()

# Initializing a model (with random weights) from the bert-base-uncased style configuration
model_random = BertModel(configuration)

# Accessing the model configuration
configuration = model_random.config

In [ ]:
%%time

# fix random seeds
fix_all_seeds()

# set up model
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on device: {}".format(device))

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
## model
# Load the pretrained model
# pretrained_model = AutoModel.from_pretrained(model_paths[0])
# Extract the embedding layer
embedding_layer = model_random.embeddings  #.word_embeddings
# new model
embedding_only_model_random = EmbeddingOnlyModel(embedding_layer)

Running on device: cuda
CPU times: user 24.4 ms, sys: 5.42 ms, total: 29.8 ms
Wall time: 144 ms


In [ ]:
# embedding sentence model
embedding_only_sentence_model_random = MyEmbeddingSentenceModel(
    embedding_only_model_random, tokenizer
)

In [ ]:
embedding_only_sentence_model_random
# embedding_only_model_random

In [ ]:
# test

In [ ]:
dummy_corpus = [
    "The sun is shining brightly in the clear blue sky.",
    "The sun is shining brightly in the clear blue sky.",
    "She made a delicious chocolate cake for dessert.",
    "They are planning a trip to Paris next summer.",
    # "The soccer team won the championship game yesterday.",
    # "He bought a new smartphone with a high-resolution camera.",
]

In [ ]:
# preprocess the input
inputs = tokenizer(
    dummy_corpus,
    padding=True,
    truncation=True,
    return_tensors="pt",
    max_length=512,
).to(device)

In [ ]:
embedding_only_sentence_model_random.encode(dummy_corpus).shape

torch.Size([4, 768])

#### Initialize model

In [ ]:
# class EmbeddingOnlyModel(torch.nn.Module):
#     def __init__(self, embeddings_module):
#         super().__init__()
#         self.embeddings = embeddings_module

#     def forward(self, input_ids, position_ids=None, inputs_embeds=None):
#         return self.embeddings(input_ids, position_ids, inputs_embeds)

In [ ]:
# from transformers import AutoModel
# import os


class EmbeddingOnlyModel(torch.nn.Module):
    def __init__(self, model_name_or_embeddings):
        super().__init__()
        if isinstance(model_name_or_embeddings, str):
            # If a string is provided, load the pretrained model and extract embeddings
            pretrained_model = AutoModel.from_pretrained(
                model_name_or_embeddings
            )
            self.embeddings = pretrained_model.embeddings
        else:
            # If embeddings are provided directly, use them
            self.embeddings = model_name_or_embeddings

    def forward(self, input_ids, position_ids=None, inputs_embeds=None):
        return self.embeddings(input_ids, position_ids, inputs_embeds)

    def save_pretrained(self, save_directory):
        os.makedirs(save_directory, exist_ok=True)
        torch.save(self.state_dict(), os.path.join(save_directory, "model.pt"))

    @classmethod
    def from_pretrained(cls, load_directory, base_model="bert-base-uncased"):
        # Load the state dict
        state_dict = torch.load(os.path.join(load_directory, "model.pt"))

        # Create a new instance of the model with a dummy model name
        # We'll replace the embeddings with the loaded state dict
        model = cls(base_model)

        # Load the state dict
        model.load_state_dict(state_dict)

        return model

In [ ]:
from transformers import BertConfig, BertModel

# Initializing a BERT bert-base-uncased style configuration
configuration = BertConfig()

# Initializing a model (with random weights) from the bert-base-uncased style configuration
model_random = BertModel(configuration)

# Accessing the model configuration
configuration = model_random.config

In [ ]:
%%time

# fix random seeds
fix_all_seeds()

# set up model
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on device: {}".format(device))

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
## model
# Load the pretrained model
# pretrained_model = AutoModel.from_pretrained(model_paths[0])
# Extract the embedding layer
embedding_layer = model_random.embeddings  # .word_embeddings
# new model
embedding_only_model_random = EmbeddingOnlyModel(embedding_layer)

Running on device: cuda
CPU times: user 29.4 ms, sys: 10.8 ms, total: 40.2 ms
Wall time: 154 ms


/gpfs01/berens/user/rgonzalesmarquez/.local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
embedding_only_model_random

EmbeddingOnlyModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)

##### Test save

In [ ]:
# Save the model and tokenizer
output_dir = Path(
    "updated_dataset/model_checkpoints/random_bert_embedding_module"
)
embedding_only_model_random.save_pretrained(variables_path / output_dir)

In [ ]:
test_model = EmbeddingOnlyModel.from_pretrained(variables_path / output_dir)

In [ ]:
test_model

EmbeddingOnlyModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)

In [ ]:
embedding_only_model_random

EmbeddingOnlyModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)

In [ ]:
# CHECK IF MODELS ARE EQUAL
def check_models_equal(original_model, loaded_model):
    # Check if state dictionaries are equal
    original_state_dict = original_model.state_dict()
    loaded_state_dict = loaded_model.state_dict()

    # This checks if all keys and tensor values are the same
    are_equal = all(
        torch.equal(original_state_dict[key], loaded_state_dict[key])
        for key in original_state_dict
    )

    print(f"Models have identical parameters: {are_equal}")

    # # You can also check individual layers if needed
    # print(torch.equal(original_model.embeddings.word_embeddings.weight,
    #                   loaded_model.embeddings.word_embeddings.weight))

#### Fine-tune it

In [ ]:
from tqdm.notebook import tqdm
from transformers.optimization import get_linear_schedule_with_warmup
from text_embeddings_src.model_stuff import (
    mean_pool,
    sep_pool,
    cls_pool,
    seventh_pool,
)
from text_embeddings_src.metrics import knn_accuracy, logistic_accuracy


def train_loop_without_eval_embedding_layer(
    model,
    loader,
    device,
    optimized_rep="av",
    n_epochs=1,
    lr=2e-5,
):
    assert optimized_rep in [
        "av",
        "cls",
        "sep",
        "7th",
    ], "Not valid `optimized_rep`. Choose from ['av', 'cls', 'sep', '7th']."

    model.to(device)

    # define layers to be used in multiple-negatives-ranking
    cos_sim = torch.nn.CosineSimilarity()
    loss_func = torch.nn.CrossEntropyLoss()
    scale = 20.0  # we multiply similarity score by this scale value, it is the inverse of the temperature
    # move layers to device
    cos_sim.to(device)
    loss_func.to(device)

    # initialize Adam optimizer
    optim = torch.optim.Adam(model.parameters(), lr=lr)

    # setup warmup for first ~10% of steps
    total_steps = len(loader) * n_epochs
    warmup_steps = int(0.1 * len(loader))
    scheduler = get_linear_schedule_with_warmup(
        optim,
        num_warmup_steps=warmup_steps,
        num_training_steps=total_steps,
    )

    losses = np.empty((n_epochs, len(loader)))
    accuracies = []
    for epoch in range(n_epochs):
        model.train()  # make sure model is in training mode
        # initialize the dataloader loop with tqdm (tqdm == progress bar)
        loop = tqdm(loader, leave=True)
        for i_batch, batch in enumerate(loop):
            # zero all gradients on each new step
            optim.zero_grad()
            # prepare batches and move all to the active device
            anchor_ids = batch[0][0].to(
                device
            )  # this are all anchor abstracts from the batch,len(anchor_ids)= len(batch)
            anchor_mask = batch[0][1].to(device)
            pos_ids = batch[1][0].to(
                device
            )  # this each positive pair from each anchor, all in one array, also len(batch)
            pos_mask = batch[1][1].to(device)
            # extract token embeddings from BERT
            a = model(anchor_ids)  # , attention_mask=anchor_mask)[
            #     0
            # ]  # all token embeddings
            p = model(pos_ids)  # , attention_mask=pos_mask)[0]

            # get the mean pooled vectors  -- put all of these ifs into a pool function (wraper) to which I pass, a, p the masks and the optimized rep
            if optimized_rep == "av":
                a = mean_pool(a, anchor_mask)
                p = mean_pool(p, pos_mask)

            elif optimized_rep == "cls":
                a = cls_pool(a, anchor_mask)
                p = cls_pool(p, pos_mask)

            elif optimized_rep == "sep":
                a = sep_pool(a, anchor_mask)
                p = sep_pool(p, pos_mask)

            elif optimized_rep == "7th":
                a = seventh_pool(a, anchor_mask)
                p = seventh_pool(p, pos_mask)

            # calculate the cosine similarities
            scores = torch.stack(
                [cos_sim(a_i.reshape(1, a_i.shape[0]), p) for a_i in a]
            )
            # get label(s) - we could define this before if confident
            # of consistent batch sizes
            labels = torch.tensor(
                range(len(scores)), dtype=torch.long, device=scores.device
            )  # I think that the labels are just the label of which pair it is. 0 for the first pair, 1 for the second...
            # my guess is that they are used in the loss to know which of the cosine similarities should be high
            # and which low

            # and now calculate the loss
            loss = loss_func(scores * scale, labels)
            losses[epoch, i_batch] = loss.item()

            # using loss, calculate gradients and then optimize
            loss.backward()
            optim.step()
            # update learning rate scheduler
            scheduler.step()
            # update the TDQM progress bar
            loop.set_description(f"Epoch {epoch}")
            loop.set_postfix(loss=loss.item())

    return model, losses

In [ ]:
# data
training_dataset = MultOverlappingSentencesPairDataset(
    iclr2024.abstract,
    tokenizer,
    device,
)

gen = torch.Generator()
gen.manual_seed(42)
training_loader = torch.utils.data.DataLoader(
    training_dataset, batch_size=64, shuffle=True, generator=gen
)

# training
(
    embedding_only_model_random_trained,
    losses,
) = train_loop_without_eval_embedding_layer(
    embedding_only_model_random,
    training_loader,
    device,
    optimized_rep="av",
    n_epochs=10,
    lr=5e-1,
)

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

In [ ]:
embedding_only_model_random_trained

EmbeddingOnlyModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)

In [ ]:
# Save the model and tokenizer
output_dir = Path(
    "updated_dataset/model_checkpoints/random_bert_embedding_module"
)
embedding_only_model_random_trained.save_pretrained(
    variables_path / output_dir
)

#### Evaluate -- all above does not need to be run

In [ ]:
# from transformers import AutoModel
import os


class EmbeddingOnlyModel(torch.nn.Module):
    def __init__(self, model_name_or_embeddings):
        super().__init__()
        if isinstance(model_name_or_embeddings, str):
            # If a string is provided, load the pretrained model and extract embeddings
            pretrained_model = AutoModel.from_pretrained(
                model_name_or_embeddings
            )
            self.embeddings = pretrained_model.embeddings
        else:
            # If embeddings are provided directly, use them
            self.embeddings = model_name_or_embeddings

    def forward(self, input_ids, position_ids=None, inputs_embeds=None):
        return self.embeddings(input_ids, position_ids, inputs_embeds)

    def save_pretrained(self, save_directory):
        os.makedirs(save_directory, exist_ok=True)
        torch.save(self.state_dict(), os.path.join(save_directory, "model.pt"))

    @classmethod
    def from_pretrained(cls, load_directory, base_model="bert-base-uncased"):
        # Load the state dict
        state_dict = torch.load(os.path.join(load_directory, "model.pt"))

        # Create a new instance of the model with a dummy model name
        # We'll replace the embeddings with the loaded state dict
        model = cls(base_model)

        # Load the state dict
        model.load_state_dict(state_dict)

        return model

In [ ]:
# fix random seeds
fix_all_seeds()

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

Running on device: cuda


/gpfs01/berens/user/rgonzalesmarquez/.local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
# load model
output_dir = Path(
    "updated_dataset/model_checkpoints/random_bert_embedding_module"
)
embedding_only_model_random_trained = EmbeddingOnlyModel.from_pretrained(
    variables_path / output_dir
)

In [ ]:
def mean_pool(token_embeds, attention_mask):
    # reshape attention_mask to cover 768-dimension embeddings
    in_mask = attention_mask.unsqueeze(-1).expand(token_embeds.size())
    # perform mean-pooling but exclude padding tokens (specified by in_mask)
    pool = torch.sum(token_embeds * in_mask, 1) / torch.clamp(
        in_mask.sum(1), min=1e-9
    )
    return pool


def cls_pool(token_embeds, attention_mask):
    ix0 = torch.arange(attention_mask.size(0))
    return token_embeds[ix0, 0, :]


import datasets


class MyEmbeddingSentenceModel:
    def __init__(self, model, tokenizer, pooling_method="av"):
        assert pooling_method in [
            "av",
            "cls",
        ], "Not valid `pooling_method`. Choose from ['av', 'cls']."

        self.tokenizer = tokenizer
        self.model = model
        self.device = (
            torch.device("cuda")
            if torch.cuda.is_available()
            else torch.device("cpu")
        )
        self.pooling_method = pooling_method

        self.model.to(self.device)

    @torch.no_grad()  # what is the difference between no_grad() and inference_mode()?
    def encode(self, input_texts, batch_size=256, **kwargs):
        inputs = self.tokenizer(
            input_texts,
            max_length=512,
            padding=True,
            truncation=True,
            return_tensors="pt",
        ).to(self.device)

        dataset = datasets.Dataset.from_dict(inputs)
        dataset.set_format(type="torch", output_all_columns=True)
        loader = torch.utils.data.DataLoader(
            dataset, batch_size=batch_size, num_workers=10
        )
        embeddings = []
        with torch.no_grad():
            for batch in loader:
                batch = {k: v.to(device) for k, v in batch.items()}
                # return batch
                # print(batch.items())
                outputs = self.model(batch["input_ids"])
                if self.pooling_method == "av":
                    embdd = mean_pool(outputs, batch["attention_mask"])
                elif self.pooling_method == "cls":
                    embdd = cls_pool(outputs, batch["attention_mask"])
                embeddings.append(embdd.detach().cpu().numpy())

        embeddings = np.vstack(embeddings)
        return embeddings  # .detach().cpu().numpy()

In [ ]:
# embedding sentence model
embedding_only_model_random_trained.to("cpu")
embedding_only_sentence_model_random = MyEmbeddingSentenceModel(
    embedding_only_model_random_trained, tokenizer
)

In [ ]:
# embedding_only_sentence_model_random.to("cpu")
embedding_only_sentence_model_random.encode(
    ["Hi my name is rita", "Hi my name is rita"], batch_size=1
).shape

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

(2, 768)

In [ ]:
tasks = mteb.get_tasks(
    tasks=[
        # "ArxivClusteringP2P",
        # "BiorxivClusteringP2P",
        # "MedrxivClusteringP2P",
        "RedditClusteringP2P",
        "TwentyNewsgroupsClustering",
        "SciDocsRR",
        "MindSmallReranking",
        "SCIDOCS",
        "ArguAna",
        "STS12",
        "STS13",
        "STS14",
        "STS15",
        "STS16",
        "STSBenchmark",
    ]
)
evaluation = mteb.MTEB(tasks=tasks)

In [ ]:
saving_path = (
    Path("embeddings_random_bert")
    / Path("updated_dataset")
    / Path("mteb_benchmark")
)
(variables_path / saving_path).mkdir(exist_ok=True)

results = evaluation.run(
    embedding_only_sentence_model_random,
    output_folder=variables_path
    / saving_path
    / f"results_embedding_module_random_bert",
)

───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

Retrieval

- SCIDOCS, s2p

- ArguAna, s2p

Clustering

...

- STSBenchmark, s2s

Reranking

- SciDocsRR, s2s

- MindSmallReranking, s2s

Clustering:   0%|                                                                       | 0/10 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possibl

### Pre-trained embedding module

#### Initialize model

In [41]:
# from transformers import AutoModel
# import os


class EmbeddingOnlyModel(torch.nn.Module):
    def __init__(self, model_name_or_embeddings):
        super().__init__()
        if isinstance(model_name_or_embeddings, str):
            # If a string is provided, load the pretrained model and extract embeddings
            pretrained_model = AutoModel.from_pretrained(
                model_name_or_embeddings
            )
            self.embeddings = pretrained_model.embeddings
        else:
            # If embeddings are provided directly, use them
            self.embeddings = model_name_or_embeddings

    def forward(self, input_ids, position_ids=None, inputs_embeds=None):
        return self.embeddings(input_ids, position_ids, inputs_embeds)

    def save_pretrained(self, save_directory):
        os.makedirs(save_directory, exist_ok=True)
        torch.save(self.state_dict(), os.path.join(save_directory, "model.pt"))

    @classmethod
    def from_pretrained(cls, load_directory, base_model="bert-base-uncased"):
        # Load the state dict
        state_dict = torch.load(os.path.join(load_directory, "model.pt"))

        # Create a new instance of the model with a dummy model name
        # We'll replace the embeddings with the loaded state dict
        model = cls(base_model)

        # Load the state dict
        model.load_state_dict(state_dict)

        return model

In [42]:
model_names = [
    "MPNet",
]


model_paths = [
    "microsoft/mpnet-base",
]

In [51]:
%%time

# fix random seeds
fix_all_seeds()

# set up model
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on device: {}".format(device))

tokenizer = AutoTokenizer.from_pretrained(model_paths[0])
## model
# Load the pretrained model
pretrained_model = AutoModel.from_pretrained(model_paths[0])
# Extract the embedding layer
embedding_layer = pretrained_model.embeddings  # .word_embeddings
# new model
embedding_only_model = EmbeddingOnlyModel(embedding_layer)

Running on device: cuda


/gpfs01/berens/user/rgonzalesmarquez/.local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of MPNetModel were not initialized from the model checkpoint at microsoft/mpnet-base and are newly initialized: ['mpnet.pooler.dense.bias', 'mpnet.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


CPU times: user 2.83 s, sys: 1.85 s, total: 4.68 s
Wall time: 1.26 s


#### Fine-tune it

In [52]:
from tqdm.notebook import tqdm
from transformers.optimization import get_linear_schedule_with_warmup
from text_embeddings_src.model_stuff import (
    mean_pool,
    sep_pool,
    cls_pool,
    seventh_pool,
)
from text_embeddings_src.metrics import knn_accuracy, logistic_accuracy


def train_loop_without_eval_embedding_layer(
    model,
    loader,
    device,
    optimized_rep="av",
    n_epochs=1,
    lr=2e-5,
):
    assert optimized_rep in [
        "av",
        "cls",
        "sep",
        "7th",
    ], "Not valid `optimized_rep`. Choose from ['av', 'cls', 'sep', '7th']."

    model.to(device)

    # define layers to be used in multiple-negatives-ranking
    cos_sim = torch.nn.CosineSimilarity()
    loss_func = torch.nn.CrossEntropyLoss()
    scale = 20.0  # we multiply similarity score by this scale value, it is the inverse of the temperature
    # move layers to device
    cos_sim.to(device)
    loss_func.to(device)

    # initialize Adam optimizer
    optim = torch.optim.Adam(model.parameters(), lr=lr)

    # setup warmup for first ~10% of steps
    total_steps = len(loader) * n_epochs
    warmup_steps = int(0.1 * len(loader))
    scheduler = get_linear_schedule_with_warmup(
        optim,
        num_warmup_steps=warmup_steps,
        num_training_steps=total_steps,
    )

    losses = np.empty((n_epochs, len(loader)))
    accuracies = []
    for epoch in range(n_epochs):
        model.train()  # make sure model is in training mode
        # initialize the dataloader loop with tqdm (tqdm == progress bar)
        loop = tqdm(loader, leave=True)
        for i_batch, batch in enumerate(loop):
            # zero all gradients on each new step
            optim.zero_grad()
            # prepare batches and move all to the active device
            anchor_ids = batch[0][0].to(
                device
            )  # this are all anchor abstracts from the batch,len(anchor_ids)= len(batch)
            anchor_mask = batch[0][1].to(device)
            pos_ids = batch[1][0].to(
                device
            )  # this each positive pair from each anchor, all in one array, also len(batch)
            pos_mask = batch[1][1].to(device)
            # extract token embeddings from BERT
            a = model(anchor_ids)  # , attention_mask=anchor_mask)[
            #     0
            # ]  # all token embeddings
            p = model(pos_ids)  # , attention_mask=pos_mask)[0]

            # get the mean pooled vectors  -- put all of these ifs into a pool function (wraper) to which I pass, a, p the masks and the optimized rep
            if optimized_rep == "av":
                a = mean_pool(a, anchor_mask)
                p = mean_pool(p, pos_mask)

            elif optimized_rep == "cls":
                a = cls_pool(a, anchor_mask)
                p = cls_pool(p, pos_mask)

            elif optimized_rep == "sep":
                a = sep_pool(a, anchor_mask)
                p = sep_pool(p, pos_mask)

            elif optimized_rep == "7th":
                a = seventh_pool(a, anchor_mask)
                p = seventh_pool(p, pos_mask)

            # calculate the cosine similarities
            scores = torch.stack(
                [cos_sim(a_i.reshape(1, a_i.shape[0]), p) for a_i in a]
            )
            # get label(s) - we could define this before if confident
            # of consistent batch sizes
            labels = torch.tensor(
                range(len(scores)), dtype=torch.long, device=scores.device
            )  # I think that the labels are just the label of which pair it is. 0 for the first pair, 1 for the second...
            # my guess is that they are used in the loss to know which of the cosine similarities should be high
            # and which low

            # and now calculate the loss
            loss = loss_func(scores * scale, labels)
            losses[epoch, i_batch] = loss.item()

            # using loss, calculate gradients and then optimize
            loss.backward()
            optim.step()
            # update learning rate scheduler
            scheduler.step()
            # update the TDQM progress bar
            loop.set_description(f"Epoch {epoch}")
            loop.set_postfix(loss=loss.item())

    return model, losses

In [53]:
# data
training_dataset = MultOverlappingSentencesPairDataset(
    iclr2024.abstract,
    tokenizer,
    device,
)

gen = torch.Generator()
gen.manual_seed(42)
training_loader = torch.utils.data.DataLoader(
    training_dataset, batch_size=64, shuffle=True, generator=gen
)

# training
(
    embedding_only_model_trained,
    losses,
) = train_loop_without_eval_embedding_layer(
    embedding_only_model,
    training_loader,
    device,
    optimized_rep="av",
    n_epochs=10,
    lr=5e-1,
)

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

In [54]:
embedding_only_model_trained

EmbeddingOnlyModel(
  (embeddings): MPNetEmbeddings(
    (word_embeddings): Embedding(30527, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)

In [55]:
# Save the model and tokenizer
output_dir = Path("updated_dataset/model_checkpoints/mpnet_embedding_module")
embedding_only_model_trained.save_pretrained(variables_path / output_dir)

#### Evaluate

In [ ]:
# from transformers import AutoModel
import os


class EmbeddingOnlyModel(torch.nn.Module):
    def __init__(self, model_name_or_embeddings):
        super().__init__()
        if isinstance(model_name_or_embeddings, str):
            # If a string is provided, load the pretrained model and extract embeddings
            pretrained_model = AutoModel.from_pretrained(
                model_name_or_embeddings
            )
            self.embeddings = pretrained_model.embeddings
        else:
            # If embeddings are provided directly, use them
            self.embeddings = model_name_or_embeddings

    def forward(self, input_ids, position_ids=None, inputs_embeds=None):
        return self.embeddings(input_ids, position_ids, inputs_embeds)

    def save_pretrained(self, save_directory):
        os.makedirs(save_directory, exist_ok=True)
        torch.save(self.state_dict(), os.path.join(save_directory, "model.pt"))

    @classmethod
    def from_pretrained(cls, load_directory, base_model="bert-base-uncased"):
        # Load the state dict
        state_dict = torch.load(os.path.join(load_directory, "model.pt"))

        # Create a new instance of the model with a dummy model name
        # We'll replace the embeddings with the loaded state dict
        model = cls(base_model)

        # Load the state dict
        model.load_state_dict(state_dict)

        return model

In [56]:
# fix random seeds
fix_all_seeds()

# tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_paths[0])

/gpfs01/berens/user/rgonzalesmarquez/.local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [57]:
# load model
output_dir = Path("updated_dataset/model_checkpoints/mpnet_embedding_module")
embedding_only_model_trained = EmbeddingOnlyModel.from_pretrained(
    variables_path / output_dir, base_model=model_paths[0]
)

Some weights of MPNetModel were not initialized from the model checkpoint at microsoft/mpnet-base and are newly initialized: ['mpnet.pooler.dense.bias', 'mpnet.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [58]:
def mean_pool(token_embeds, attention_mask):
    # reshape attention_mask to cover 768-dimension embeddings
    in_mask = attention_mask.unsqueeze(-1).expand(token_embeds.size())
    # perform mean-pooling but exclude padding tokens (specified by in_mask)
    pool = torch.sum(token_embeds * in_mask, 1) / torch.clamp(
        in_mask.sum(1), min=1e-9
    )
    return pool


def cls_pool(token_embeds, attention_mask):
    ix0 = torch.arange(attention_mask.size(0))
    return token_embeds[ix0, 0, :]


import datasets


class MyEmbeddingSentenceModel:
    def __init__(self, model, tokenizer, pooling_method="av"):
        assert pooling_method in [
            "av",
            "cls",
        ], "Not valid `pooling_method`. Choose from ['av', 'cls']."

        self.tokenizer = tokenizer
        self.model = model
        self.device = (
            torch.device("cuda")
            if torch.cuda.is_available()
            else torch.device("cpu")
        )
        self.pooling_method = pooling_method

        self.model.to(self.device)

    @torch.no_grad()  # what is the difference between no_grad() and inference_mode()?
    def encode(self, input_texts, batch_size=256, **kwargs):
        inputs = self.tokenizer(
            input_texts,
            max_length=512,
            padding=True,
            truncation=True,
            return_tensors="pt",
        ).to(self.device)

        dataset = datasets.Dataset.from_dict(inputs)
        dataset.set_format(type="torch", output_all_columns=True)
        loader = torch.utils.data.DataLoader(
            dataset, batch_size=batch_size, num_workers=10
        )
        embeddings = []
        with torch.no_grad():
            for batch in loader:
                batch = {k: v.to(device) for k, v in batch.items()}
                # return batch
                # print(batch.items())
                outputs = self.model(batch["input_ids"])
                if self.pooling_method == "av":
                    embdd = mean_pool(outputs, batch["attention_mask"])
                elif self.pooling_method == "cls":
                    embdd = cls_pool(outputs, batch["attention_mask"])
                embeddings.append(embdd.detach().cpu().numpy())

        embeddings = np.vstack(embeddings)
        return embeddings  # .detach().cpu().numpy()

In [59]:
# embedding sentence model
embedding_only_model_trained.to("cpu")
embedding_only_sentence_model = MyEmbeddingSentenceModel(
    embedding_only_model_trained, tokenizer
)

In [60]:
tasks = mteb.get_tasks(
    tasks=[
        "ArxivClusteringP2P",
        "BiorxivClusteringP2P",
        "MedrxivClusteringP2P",
        "RedditClusteringP2P",
        "TwentyNewsgroupsClustering",
        "SciDocsRR",
        "MindSmallReranking",
        "SCIDOCS",
        "ArguAna",
        "STS12",
        "STS13",
        "STS14",
        "STS15",
        "STS16",
        "STSBenchmark",
    ]
)
evaluation = mteb.MTEB(tasks=tasks)

In [61]:
saving_path = (
    Path("embeddings_" + model_names[0].lower())
    / Path("updated_dataset")
    / Path("mteb_benchmark")
)
(variables_path / saving_path).mkdir(exist_ok=True)

results = evaluation.run(
    embedding_only_sentence_model_random,
    output_folder=variables_path
    / saving_path
    / f"results_embedding_module_{model_names[0].lower()}",
)

───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

Retrieval

- SCIDOCS, s2p

- ArguAna, s2p

Clustering

...

- STSBenchmark, s2s

Reranking

- SciDocsRR, s2s

- MindSmallReranking, s2s

Clustering:   0%|                                                                       | 0/31 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possibl

### Random embedding layer

#### Test set up model

##### Way 2 -- good one

In [68]:
def mean_pool(token_embeds, attention_mask):
    # reshape attention_mask to cover 768-dimension embeddings
    in_mask = attention_mask.unsqueeze(-1).expand(token_embeds.size())
    # perform mean-pooling but exclude padding tokens (specified by in_mask)
    pool = torch.sum(token_embeds * in_mask, 1) / torch.clamp(
        in_mask.sum(1), min=1e-9
    )
    return pool


def cls_pool(token_embeds, attention_mask):
    ix0 = torch.arange(attention_mask.size(0))
    return token_embeds[ix0, 0, :]


import datasets


class MyEmbeddingSentenceModel:
    def __init__(self, model, tokenizer, pooling_method="av"):
        assert pooling_method in [
            "av",
            "cls",
        ], "Not valid `pooling_method`. Choose from ['av', 'cls']."

        self.tokenizer = tokenizer
        self.model = model
        self.device = (
            torch.device("cuda")
            if torch.cuda.is_available()
            else torch.device("cpu")
        )
        self.pooling_method = pooling_method

        self.model.to(self.device)

    @torch.no_grad()  # what is the difference between no_grad() and inference_mode()?
    def encode(self, input_texts, batch_size=None, **kwargs):
        inputs = self.tokenizer(
            input_texts,
            max_length=512,
            padding=True,
            truncation=True,
            return_tensors="pt",
        ).to(self.device)

        dataset = datasets.Dataset.from_dict(inputs)
        dataset.set_format(type="torch", output_all_columns=True)
        loader = torch.utils.data.DataLoader(
            dataset, batch_size=batch_size, num_workers=10
        )
        embeddings = []
        with torch.no_grad():
            for batch in loader:
                batch = {k: v.to(device) for k, v in batch.items()}
                outputs = self.model(batch["input_ids"])
                if self.pooling_method == "av":
                    embdd = mean_pool(outputs, batch["attention_mask"])
                elif self.pooling_method == "cls":
                    embdd = cls_pool(outputs, batch["attention_mask"])
                embeddings.append(embdd.detach().cpu().numpy())

        embeddings = np.vstack(embeddings)
        return embeddings 

In [ ]:
# from transformers import AutoModel
# import os


class EmbeddingOnlyModel(torch.nn.Module):
    def __init__(self, model_name_or_embeddings):
        super().__init__()
        if isinstance(model_name_or_embeddings, str):
            # If a string is provided, load the pretrained model and extract embeddings
            pretrained_model = AutoModel.from_pretrained(
                model_name_or_embeddings
            )
            self.embeddings = pretrained_model.embeddings.word_embeddings
        else:
            # If embeddings are provided directly, use them
            self.embeddings = model_name_or_embeddings

    def forward(self, input_ids):
        return self.embeddings(input_ids)

    def save_pretrained(self, save_directory):
        os.makedirs(save_directory, exist_ok=True)
        torch.save(self.state_dict(), os.path.join(save_directory, "model.pt"))

    @classmethod
    def from_pretrained(cls, load_directory, base_model="bert-base-uncased"):
        # Load the state dict
        state_dict = torch.load(os.path.join(load_directory, "model.pt"))

        # Create a new instance of the model with a dummy model name
        # We'll replace the embeddings with the loaded state dict
        model = cls(base_model)

        # Load the state dict
        model.load_state_dict(state_dict)

        return model

In [70]:
from transformers import BertConfig, BertModel

# Initializing a BERT bert-base-uncased style configuration
configuration = BertConfig()

# Initializing a model (with random weights) from the bert-base-uncased style configuration
model_random = BertModel(configuration)

# Accessing the model configuration
configuration = model_random.config

In [71]:
%%time

# fix random seeds
fix_all_seeds()

# set up model
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on device: {}".format(device))

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
## model
# Load the pretrained model
# pretrained_model = AutoModel.from_pretrained(model_paths[0])
# Extract the embedding layer
embedding_layer = model_random.embeddings.word_embeddings
# new model
embedding_only_model_random = EmbeddingOnlyModel(embedding_layer)

Running on device: cuda
CPU times: user 35.4 ms, sys: 8.64 ms, total: 44.1 ms
Wall time: 155 ms


In [72]:
# embedding sentence model
embedding_only_sentence_model_random = MyEmbeddingSentenceModel(
    embedding_only_model_random, tokenizer
)

In [73]:
embedding_only_sentence_model_random
# embedding_only_model_random

In [74]:
# test

In [75]:
dummy_corpus = [
    "The sun is shining brightly in the clear blue sky.",
    "The sun is shining brightly in the clear blue sky.",
    "She made a delicious chocolate cake for dessert.",
    "They are planning a trip to Paris next summer.",
    # "The soccer team won the championship game yesterday.",
    # "He bought a new smartphone with a high-resolution camera.",
]

In [76]:
# preprocess the input
inputs = tokenizer(
    dummy_corpus,
    padding=True,
    truncation=True,
    return_tensors="pt",
    max_length=512,
).to(device)

In [77]:
embedding_only_sentence_model_random.encode(dummy_corpus).shape

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

(4, 13)

#### Initialize model

In [78]:
# from transformers import AutoModel
# import os


class EmbeddingOnlyModel(torch.nn.Module):
    def __init__(self, model_name_or_embeddings):
        super().__init__()
        if isinstance(model_name_or_embeddings, str):
            # If a string is provided, load the pretrained model and extract embeddings
            pretrained_model = AutoModel.from_pretrained(
                model_name_or_embeddings
            )
            self.embeddings = pretrained_model.embeddings.word_embeddings
        else:
            # If embeddings are provided directly, use them
            self.embeddings = model_name_or_embeddings

    def forward(self, input_ids):
        return self.embeddings(input_ids)

    def save_pretrained(self, save_directory):
        os.makedirs(save_directory, exist_ok=True)
        torch.save(self.state_dict(), os.path.join(save_directory, "model.pt"))

    @classmethod
    def from_pretrained(cls, load_directory, base_model="bert-base-uncased"):
        # Load the state dict
        state_dict = torch.load(os.path.join(load_directory, "model.pt"))

        # Create a new instance of the model with a dummy model name
        # We'll replace the embeddings with the loaded state dict
        model = cls(base_model)

        # Load the state dict
        model.load_state_dict(state_dict)

        return model

In [79]:
from transformers import BertConfig, BertModel

# Initializing a BERT bert-base-uncased style configuration
configuration = BertConfig()

# Initializing a model (with random weights) from the bert-base-uncased style configuration
model_random = BertModel(configuration)

# Accessing the model configuration
configuration = model_random.config

In [80]:
%%time

# fix random seeds
fix_all_seeds()

# set up model
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on device: {}".format(device))

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
## model
# Load the pretrained model
# pretrained_model = AutoModel.from_pretrained(model_paths[0])
# Extract the embedding layer
embedding_layer = model_random.embeddings.word_embeddings
# new model
embedding_only_model_random = EmbeddingOnlyModel(embedding_layer)

Running on device: cuda
CPU times: user 38.6 ms, sys: 11.8 ms, total: 50.4 ms
Wall time: 161 ms


In [81]:
embedding_only_model_random

EmbeddingOnlyModel(
  (embeddings): Embedding(30522, 768, padding_idx=0)
)

##### Test save

In [84]:
# Save the model and tokenizer
output_dir = Path(
    "updated_dataset/model_checkpoints/random_bert_embedding_layer"
)
embedding_only_model_random.save_pretrained(variables_path / output_dir)

In [85]:
test_model = EmbeddingOnlyModel.from_pretrained(variables_path / output_dir)

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [86]:
test_model

EmbeddingOnlyModel(
  (embeddings): Embedding(30522, 768, padding_idx=0)
)

In [87]:
embedding_only_model_random

EmbeddingOnlyModel(
  (embeddings): Embedding(30522, 768, padding_idx=0)
)

In [88]:
# CHECK IF MODELS ARE EQUAL
def check_models_equal(original_model, loaded_model):
    # Check if state dictionaries are equal
    original_state_dict = original_model.state_dict()
    loaded_state_dict = loaded_model.state_dict()

    # This checks if all keys and tensor values are the same
    are_equal = all(
        torch.equal(original_state_dict[key], loaded_state_dict[key])
        for key in original_state_dict
    )

    print(f"Models have identical parameters: {are_equal}")

    # # You can also check individual layers if needed
    # print(torch.equal(original_model.embeddings.word_embeddings.weight,
    #                   loaded_model.embeddings.word_embeddings.weight))

In [89]:
check_models_equal(embedding_only_model_random, test_model)

Models have identical parameters: True


#### Fine-tune it

In [90]:
from tqdm.notebook import tqdm
from transformers.optimization import get_linear_schedule_with_warmup
from text_embeddings_src.model_stuff import (
    mean_pool,
    sep_pool,
    cls_pool,
    seventh_pool,
)
from text_embeddings_src.metrics import knn_accuracy, logistic_accuracy


def train_loop_without_eval_embedding_layer(
    model,
    loader,
    device,
    optimized_rep="av",
    n_epochs=1,
    lr=2e-5,
):
    assert optimized_rep in [
        "av",
        "cls",
        "sep",
        "7th",
    ], "Not valid `optimized_rep`. Choose from ['av', 'cls', 'sep', '7th']."

    model.to(device)

    # define layers to be used in multiple-negatives-ranking
    cos_sim = torch.nn.CosineSimilarity()
    loss_func = torch.nn.CrossEntropyLoss()
    scale = 20.0  # we multiply similarity score by this scale value, it is the inverse of the temperature
    # move layers to device
    cos_sim.to(device)
    loss_func.to(device)

    # initialize Adam optimizer
    optim = torch.optim.Adam(model.parameters(), lr=lr)

    # setup warmup for first ~10% of steps
    total_steps = len(loader) * n_epochs
    warmup_steps = int(0.1 * len(loader))
    scheduler = get_linear_schedule_with_warmup(
        optim,
        num_warmup_steps=warmup_steps,
        num_training_steps=total_steps,
    )

    losses = np.empty((n_epochs, len(loader)))
    accuracies = []
    for epoch in range(n_epochs):
        model.train()  # make sure model is in training mode
        # initialize the dataloader loop with tqdm (tqdm == progress bar)
        loop = tqdm(loader, leave=True)
        for i_batch, batch in enumerate(loop):
            # zero all gradients on each new step
            optim.zero_grad()
            # prepare batches and move all to the active device
            anchor_ids = batch[0][0].to(
                device
            )  # this are all anchor abstracts from the batch,len(anchor_ids)= len(batch)
            anchor_mask = batch[0][1].to(device)
            pos_ids = batch[1][0].to(
                device
            )  # this each positive pair from each anchor, all in one array, also len(batch)
            pos_mask = batch[1][1].to(device)
            # extract token embeddings from BERT
            a = model(anchor_ids)  # , attention_mask=anchor_mask)[
            #     0
            # ]  # all token embeddings
            p = model(pos_ids)  # , attention_mask=pos_mask)[0]

            # get the mean pooled vectors  -- put all of these ifs into a pool function (wraper) to which I pass, a, p the masks and the optimized rep
            if optimized_rep == "av":
                a = mean_pool(a, anchor_mask)
                p = mean_pool(p, pos_mask)

            elif optimized_rep == "cls":
                a = cls_pool(a, anchor_mask)
                p = cls_pool(p, pos_mask)

            elif optimized_rep == "sep":
                a = sep_pool(a, anchor_mask)
                p = sep_pool(p, pos_mask)

            elif optimized_rep == "7th":
                a = seventh_pool(a, anchor_mask)
                p = seventh_pool(p, pos_mask)

            # calculate the cosine similarities
            scores = torch.stack(
                [cos_sim(a_i.reshape(1, a_i.shape[0]), p) for a_i in a]
            )
            # get label(s) - we could define this before if confident
            # of consistent batch sizes
            labels = torch.tensor(
                range(len(scores)), dtype=torch.long, device=scores.device
            )  # I think that the labels are just the label of which pair it is. 0 for the first pair, 1 for the second...
            # my guess is that they are used in the loss to know which of the cosine similarities should be high
            # and which low

            # and now calculate the loss
            loss = loss_func(scores * scale, labels)
            losses[epoch, i_batch] = loss.item()

            # using loss, calculate gradients and then optimize
            loss.backward()
            optim.step()
            # update learning rate scheduler
            scheduler.step()
            # update the TDQM progress bar
            loop.set_description(f"Epoch {epoch}")
            loop.set_postfix(loss=loss.item())

    return model, losses

In [91]:
# data
training_dataset = MultOverlappingSentencesPairDataset(
    iclr2024.abstract,
    tokenizer,
    device,
)

gen = torch.Generator()
gen.manual_seed(42)
training_loader = torch.utils.data.DataLoader(
    training_dataset, batch_size=64, shuffle=True, generator=gen
)

# training
(
    embedding_only_model_random_trained,
    losses,
) = train_loop_without_eval_embedding_layer(
    embedding_only_model_random,
    training_loader,
    device,
    optimized_rep="av",
    n_epochs=10,
    lr=5e-1,
)

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

  0%|          | 0/368 [00:00<?, ?it/s]

In [92]:
embedding_only_model_random_trained

EmbeddingOnlyModel(
  (embeddings): Embedding(30522, 768, padding_idx=0)
)

In [93]:
# Save the model and tokenizer
output_dir = Path(
    "updated_dataset/model_checkpoints/random_bert_embedding_layer"
)
embedding_only_model_random_trained.save_pretrained(
    variables_path / output_dir
)

#### Evaluate -- all above does not need to be run

In [94]:
# from transformers import AutoModel
# import os


class EmbeddingOnlyModel(torch.nn.Module):
    def __init__(self, model_name_or_embeddings):
        super().__init__()
        if isinstance(model_name_or_embeddings, str):
            # If a string is provided, load the pretrained model and extract embeddings
            pretrained_model = AutoModel.from_pretrained(
                model_name_or_embeddings
            )
            self.embeddings = pretrained_model.embeddings.word_embeddings
        else:
            # If embeddings are provided directly, use them
            self.embeddings = model_name_or_embeddings

    def forward(self, input_ids):
        return self.embeddings(input_ids)

    def save_pretrained(self, save_directory):
        os.makedirs(save_directory, exist_ok=True)
        torch.save(self.state_dict(), os.path.join(save_directory, "model.pt"))

    @classmethod
    def from_pretrained(cls, load_directory, base_model="bert-base-uncased"):
        # Load the state dict
        state_dict = torch.load(os.path.join(load_directory, "model.pt"))

        # Create a new instance of the model with a dummy model name
        # We'll replace the embeddings with the loaded state dict
        model = cls(base_model)

        # Load the state dict
        model.load_state_dict(state_dict)

        return model

In [95]:
# fix random seeds
fix_all_seeds()

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [97]:
# load model
output_dir = Path(
    "updated_dataset/model_checkpoints/random_bert_embedding_layer"
)
embedding_only_model_random_trained = EmbeddingOnlyModel.from_pretrained(
    variables_path / output_dir
)

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [98]:
def mean_pool(token_embeds, attention_mask):
    # reshape attention_mask to cover 768-dimension embeddings
    in_mask = attention_mask.unsqueeze(-1).expand(token_embeds.size())
    # perform mean-pooling but exclude padding tokens (specified by in_mask)
    pool = torch.sum(token_embeds * in_mask, 1) / torch.clamp(
        in_mask.sum(1), min=1e-9
    )
    return pool


def cls_pool(token_embeds, attention_mask):
    ix0 = torch.arange(attention_mask.size(0))
    return token_embeds[ix0, 0, :]


import datasets


class MyEmbeddingSentenceModel:
    def __init__(self, model, tokenizer, pooling_method="av"):
        assert pooling_method in [
            "av",
            "cls",
        ], "Not valid `pooling_method`. Choose from ['av', 'cls']."

        self.tokenizer = tokenizer
        self.model = model
        self.device = (
            torch.device("cuda")
            if torch.cuda.is_available()
            else torch.device("cpu")
        )
        self.pooling_method = pooling_method

        self.model.to(self.device)

    @torch.no_grad()  # what is the difference between no_grad() and inference_mode()?
    def encode(self, input_texts, batch_size=None, **kwargs):
        inputs = self.tokenizer(
            input_texts,
            max_length=512,
            padding=True,
            truncation=True,
            return_tensors="pt",
        ).to(self.device)

        dataset = datasets.Dataset.from_dict(inputs)
        dataset.set_format(type="torch", output_all_columns=True)
        loader = torch.utils.data.DataLoader(
            dataset, batch_size=batch_size, num_workers=10
        )
        embeddings = []
        with torch.no_grad():
            for batch in loader:
                batch = {k: v.to(device) for k, v in batch.items()}
                outputs = self.model(batch["input_ids"])
                if self.pooling_method == "av":
                    embdd = mean_pool(outputs, batch["attention_mask"])
                elif self.pooling_method == "cls":
                    embdd = cls_pool(outputs, batch["attention_mask"])
                embeddings.append(embdd.detach().cpu().numpy())

        embeddings = np.vstack(embeddings)
        return embeddings

In [99]:
# embedding sentence model
embedding_only_model_random_trained.to("cpu")
embedding_only_sentence_model_random = MyEmbeddingSentenceModel(
    embedding_only_model_random_trained, tokenizer
)

In [102]:
tasks = mteb.get_tasks(
    tasks=[
        "ArxivClusteringP2P",
        "BiorxivClusteringP2P",
        "MedrxivClusteringP2P",
        "RedditClusteringP2P",
        # "TwentyNewsgroupsClustering",
        "StackExchangeClusteringP2P",
        "SciDocsRR",
        "MindSmallReranking",
        "SCIDOCS",
        "ArguAna",
        "STS12",
        "STS13",
        "STS14",
        "STS15",
        "STS16",
        "STSBenchmark",
    ]
)
evaluation = mteb.MTEB(tasks=tasks)

In [103]:
saving_path = (
    Path("embeddings_random_bert")
    / Path("updated_dataset")
    / Path("mteb_benchmark")
)
(variables_path / saving_path).mkdir(exist_ok=True)

results = evaluation.run(
    embedding_only_sentence_model_random,
    output_folder=variables_path
    / saving_path
    / f"results_embedding_layer_random_bert",
)

───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

Retrieval

- SCIDOCS, s2p

- ArguAna, s2p

STS

...

- RedditClusteringP2P, p2p

Reranking

- SciDocsRR, s2s

- MindSmallReranking, s2s

Clustering:   0%|                                                                       | 0/31 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possibl